In [20]:
import FuncionesPreProcess as fp
import os
import cv2
import numpy as np 

In [21]:
def sacarDiferencia(carpeta, newCarpeta):
    # Crear la carpeta de imágenes de diferencia si no existe
    if not os.path.exists(newCarpeta):
        os.makedirs(newCarpeta)

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(carpeta)

    # Iterar sobre los archivos
    for archivo in archivos:
        # Comprobar si es un archivo de imagen
        if archivo.endswith(".jpg") or archivo.endswith(".png"):
            # Ruta completa de la imagen
            ruta_imagen = os.path.join(carpeta, archivo)

            # Cargar la imagen
            imagen = cv2.imread(ruta_imagen, 0)
            imagen = fp.normalize(imagen)

            # Eliminación de ruido con filtrado de media
            imagen = fp.medianBlur(imagen, 3)

            # Mejora de la nitidez utilizando filtrado bilateral
            imagen = fp.bilateral_filter(imagen, 9, 40, 100)

            # Obtener dimensiones de la imagen
            alto, ancho = imagen.shape

            # Dividir la imagen en mitades
            mitad_izquierda = imagen[:, :ancho // 2]
            mitad_derecha = imagen[:, ancho // 2:]

            # Aplicar efecto espejo vertical a la mitad derecha
            mitad_derecha_espejo = fp.espejoY(mitad_derecha)
            
            diferencia = np.abs(mitad_izquierda.astype(int) - mitad_derecha_espejo.astype(int))

            # Convertir la imagen de diferencia a CV_8U
            diferencia = fp.convertAbs(diferencia)
            
            # Aplicar el filtro bilateral
            diferencia = fp.bilateral_filter(diferencia, 9, 300, 2000)

            # Guardar la imagen de diferencia en escala de grises
            nombre_grafica = "mitad_" + archivo
            ruta_grafica = os.path.join(newCarpeta, nombre_grafica)
            cv2.imwrite(ruta_grafica, diferencia)


In [22]:
'''Esta parte solo se ejecuta una vez'''
#sacarDiferencia('yes_nuevo/','./carpeta_MitadesYesM/')

#sacarDiferencia('no_nuevo/','./carpeta_MitadesNoM/')

In [23]:
data_yes = fp.vectorizar('carpeta_MitadesYesM/', 1)
data_no = fp.vectorizar('carpeta_MitadesNoM/', 0)

In [49]:
import FuncionesIA as fia

In [25]:
allData = fia.mergeData(data_yes, data_no)
print(allData.shape)

(207, 6252)


In [26]:
def LOO_Knn(allData):
    labelsPredictedKNN = []
    labelReal = allData[:, 6250]
    for i in range(allData.shape[0]):
        dataTrain = fia.exclude(allData, i)
        ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)  


        dataTest = allData[i, :]
        dataTest = dataTest.reshape(1, 6252)
        ImgsTest = np.array(dataTest[:, :-2], dtype=int)
        LabelsTest = np.array(dataTest[:, 6250], dtype=int)
        NameImageTest = np.array(dataTest[:, 6251], dtype=str)

        knn = fia.KNNClassifier(1)
        knn.fit(ImgsTrain, LabelsTrain)
        prediction = knn.predict(ImgsTest)
        
        labelsPredictedKNN.append(prediction[0])
    labelsPredictedKNN = np.array(labelsPredictedKNN)  
    return(labelsPredictedKNN)

In [27]:
def LOO_KMeans(allData):
    labelsPre = []
    labelsR = []
    labelsPlus = []
    for i in range(allData.shape[0]):
        dataTrain = fia.exclude(allData, i)
        ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)  


        dataTest = allData[i, :]
        dataTest = dataTest.reshape(1, 6252)
        ImgsTest = np.array(dataTest[:, :-2], dtype=int)
        LabelsTest = np.array(dataTest[:, 6250], dtype=int)
        NameImageTest = np.array(dataTest[:, 6251], dtype=str)

        iterations, k=2, 2

        kmeansPre = fia.KMeans(k,iterations)
        kmeansR = fia.KMeans(k,iterations)
        kmeansPlus = fia.KMeans(k,iterations)
        
        # Ajustar el modelo a los datos
        kmeansPre.fitPreprocess(ImgsTrain, LabelsTrain)
        kmeansR.fitRANDOM(ImgsTrain)
        kmeansPlus.fitKplus(ImgsTrain)
        
        # Obtener las etiquetas asignadas a cada punto
        predicted_labelsPre = kmeansPre.predict(ImgsTest)
        predicted_labelsR = kmeansR.predict(ImgsTest)
        predicted_labelsPlus = kmeansPlus.predict(ImgsTest)
        
        labelsPre.append(predicted_labelsPre[0])
        labelsR.append(predicted_labelsR[0])
        labelsPlus.append(predicted_labelsPlus[0])
    
    accuracyPre = np.mean(labelsPre == np.array(allData[:, 6250], dtype=int))
    accuracyR = np.mean(labelsR == np.array(allData[:, 6250], dtype=int))
    accuracyPlus = np.mean(labelsPlus == np.array(allData[:, 6250], dtype=int))
    
    print(accuracyPre,accuracyR,accuracyPlus)
    

In [28]:
LOO_KMeans(allData)

0.6956521739130435 0.5458937198067633 0.5072463768115942


In [29]:
predictions = LOO_Knn(allData)

In [30]:
accuracy = np.mean(predictions == np.array(allData[:, 6250], dtype=int))
print(accuracy)

0.7922705314009661


In [50]:
NamePredicction =  fia.mergeData(predictions,allData[:, 6251])
NamePredicction = NamePredicction.reshape(2, 207)

In [52]:
NamePredictionYES= fia.positiveData(NamePredicction)
print(NamePredictionYES)

AttributeError: module 'FuncionesIA' has no attribute 'positiveData'

In [ ]:
archivos = os.listdir('mergedImages/')
for archivo in archivos:
    